In [39]:
import pandas as pd
import numpy as np
import implicit
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse as sp



This notebook contains two models for the recommendation system.  The Implicit and lightfm libraries will be used for the models.  The Implicit model was chosen because the type of data we have for the reccomender is implicit meaning, only interactions are stored and not user supplied ratings as in explicit data.  Implicit data, while more difficult to interpret, is generated in much larger amounts than explicit.  While implicit data is more vague, the volume of data should help the model perform better.  The lightfm library was chosen for its capability of constructing hybrid models where user and item features can be supplied.  Including user and content features helps to mitigate the cold start problem where the user/item matrix has difficulty making adequate recommendation due to its sparsity.  Constructing a hybrid model is beyond the scope of this project.  However, lightfm was chosen in the event the project is further developed into a hybrid model.

## Creating user/item matrix

In [58]:
#read in data
le = pd.read_csv('nprs_final.csv',usecols=['user_id','track_id','created_at'])

In [59]:
le.head()

,user_id,track_id,created_at
0,81496937,cd52b3e5b51da29e5893dba82a418a4b,2014-01-01 05:54:21
1,81496937,cd52b3e5b51da29e5893dba82a418a4b,2014-02-19 11:21:51
2,81496937,cd52b3e5b51da29e5893dba82a418a4b,2014-03-02 15:28:20
3,81496937,cd52b3e5b51da29e5893dba82a418a4b,2014-03-04 01:32:31
4,81496937,cd52b3e5b51da29e5893dba82a418a4b,2014-04-07 20:20:46


In [60]:
les = le.sort_values(by='created_at')

In [61]:
les.head()

,user_id,track_id,created_at
0,81496937,cd52b3e5b51da29e5893dba82a418a4b,2014-01-01 05:54:21
20036,2205686924,da3110a77b724072b08f231c9d6f7534,2014-01-01 05:54:22
631443,97675221,33f95122281f76e7134f9cbea3be980f,2014-01-01 05:54:24
753822,452285741,8bd5206b84c968eda0af8bc86d6ab1d1,2014-01-01 05:54:25
836237,65086276,23ced06ca57d37fa749b1595bc7ed1a4,2014-01-01 05:54:28


In [62]:
#assign each event a count of 1
les['count'] = 1

In [64]:
#group by user/item id and sum counts
rating = les.groupby(['user_id','track_id'],as_index=False)['count'].sum()

In [65]:
rating.head()

,user_id,track_id,count
0,10221,03b403c0ecede5105ac9f3880a7a480a,3
1,10221,09553c80ea087b5885436c627f00f482,1
2,10221,1c23db8ed72b8a1e44f4b5af9d7739bf,1
3,10221,299c9ab0f09d8dc281deaaad3260853e,5
4,10221,34619be190aa19b20492e13da0cc1e45,1


In [66]:
#unique list of users
users = list(rating.user_id.unique())
#unique list of songs
songs = list(rating.track_id.unique())
#count of interactions per user/item
conf = list(rating['count'])

In [70]:
#assign sequential user and item indicies for sparse matrix
#assign category codes to user
rows = rating.user_id.astype('category').cat.codes
#assign category code to item
cols = rating.track_id.astype('category').cat.codes

In [71]:
#create sparse matrix of users/items with interaction count as entry
smat = sp.csr_matrix((conf,(rows,cols)),shape=(len(users),len(songs)))

In [74]:
#check properties of sparse matrix
smat

<21220x81343 sparse matrix of type '<class 'numpy.intc'>'
	with 1053887 stored elements in Compressed Sparse Row format>

In [76]:
matrix_size = smat.shape[0]*smat.shape[1] # Number of possible interactions in the matrix
num_listens = len(smat.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
print(f'Sparsity of the user/item matrix is {sparsity}')

Sparsity of the user/item matrix is 99.93894398121415


## Train/test split

In many machine learning applications, splitting the train and test sets is done by randomly sampling rows to set aside for training.  However, this method is inadequate for collaborative filtering because the interactions are needed to complete the matrix factorization.  Another method is to mask interactions in the training set, complete the matrix factorization, and compare the dot product of the latent vectors at the indices in which interactions were masked, to the unmasked interactions.  In this sense, the training set is altered, and the test set is the original user/item matrix.  This can be done by finding the nonzero interactions in the training matrix, set a percentage of them to 0, and record the indices. This is done in the following function.  The function outputs the train set, test set, and list of users who had an item masked.

In [79]:
import random

In [80]:
def make_train(ratings, pct_test = 0.2):
    # copy original data for test set
    test_set = ratings.copy()  
    # set confidnece levels to 1: binary matrix
    test_set[test_set != 0] = 1 
    # Copy original data to mask users for train set
    training_set = ratings.copy()  
    # Find the interaction indices
    nonzero_inds = training_set.nonzero() 
    # Zip non-zero entities into a list to sample from
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    # Set the random seed 
    random.seed(0) 
    # Round the number of samples needed to the nearest integer
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) 
    # Sample user-item pairs without replacement
    samples = random.sample(nonzero_pairs, num_samples) 
    # Get the user row indices
    user_inds = [index[0] for index in samples] # Get the user row indices
    # Get the item column indices
    item_inds = [index[1] for index in samples] 
    # Assign all of the randomly chosen user-item pairs to zero
    training_set[user_inds, item_inds] = 0 
    # Get rid of zeros in sparse array storage after update to save space
    training_set.eliminate_zeros() 
    # Output the unique list of user rows that were altered
    return training_set, test_set, list(set(user_inds))


In [243]:
#call function to create train/test split and masked user list 
train, test, masked_users = make_train(smat,pct_test = 0.2)

In [253]:
#check properties of the train set
train

<21220x81343 sparse matrix of type '<class 'numpy.intc'>'
	with 843109 stored elements in Compressed Sparse Row format>

In [254]:
#check properties of test set
test

<21220x81343 sparse matrix of type '<class 'numpy.intc'>'
	with 1053887 stored elements in Compressed Sparse Row format>

Inspecting the propeties of each matrix shows that the dimensions are the same, with 20% less stored elements in the train set (items that were masked).  We an now fit the data to the alternating least sqaures model implemented in the implicit library.

## Alternating Least Squares with Implicit library

Next we construct the alternating least sqaures model using the implicit library.  The tuning parameter, alpha, is set to 40 and the model is called with the number of factos set to 20.  The output of the model will be the user and item vectors associated with the latent features of the matrix factorization.  These vectors are then used to make predictions by taking the dot product at the user indices where interactions were masked.

In [83]:
#tuning parameter for confidence value
alpha = 40
user_vecs, item_vecs = implicit.alternating_least_squares((train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                          iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


In [94]:
#verify user vector shape
user_vecs.shape              

(21220, 20)

In [261]:
    #verify item vector shape
    item_vecs.shape

(81343, 20)

## Model Evaluation with AUC

Now that we have the latent user and item feature vectors the next step is to make predictions by taking the dot product at the indices that were masked, and compare the results to binarixed test set.  As mentioned previously, access to the artist names and song titles are unavailable.  Because of this, there is no way to qualitatively access how the recommendation system is performing.  We will rely on the mean area under the curve metric for the interactions that were masked, and as comparison the popularity auc.

In [127]:
from sklearn import metrics

In [128]:
def auc_score(predictions, test):
    
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)  

In [225]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    
     #empty lists to store scores
    store_auc = [] 
    popularity_auc = [] 
    
    # Get sum of item iteractions to find most popular
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1)
    #Get latent item vectors
    item_vecs = predictions[1]
    for user in altered_users:
        
        #training set row, 1 dimensional vector
        training_row = training_set[user,:].toarray().reshape(-1) 
        # Find where the interaction had not yet occurred
        zero_inds = np.where(training_row == 0) 
        # Get latent vectors for each user
        user_vec = predictions[0][user,:]
        #make predictions get only the indices with no interaction
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Select the binary interaction pairs from original data aligned with pairs from training
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
         # Get the item popularity for our chosen items
        pop = pop_items[zero_inds]
        #append auc scores to list
        store_auc.append(auc_score(pred, actual))
        #append popularity auc scores to list
        popularity_auc.append(auc_score(pop, actual)) 
    
    # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
  

In [131]:
calc_mean_auc(train, masked_users, 
              [sp.csr_matrix(user_vecs), sp.csr_matrix(item_vecs.T)],test)

(0.916, 0.875)

We can see from the above results that the recommendation system outperformed the popularity model by about 4%.  Ideally, the parameters of the model would be searched to fine the optimal parameters. However, this is computationally expensive on the current system due to having to loop through the users with masked interactions.  To improve the model performance it would be better to add user/item features to add content metadata.  With the auc score for the ALS model being above 90%, this can be considered a good stopping point fo the ALS model, and we can pivot to constructing a model in lightfm that has the capability to create a hybrid model that includes matrix factorization and content modelling.  As mentioned before, we will not be creating a hybrid model, rather we can create a standard matrix factorization model in lightfm simply by not supplying any user or item features.  This will be used to compare the performance with the ALS model.

## LightFM model

In able to compare to the ALS model to another, we build a model in lightfm to compare the performance.  Using the lightfm model will also familiarize us to the library in the event the project is extended to creating a hybrid model in lightfm.  Lightfm provides utilities to make model fitting easy.  A model and parameters are defined, in addition to using the libraries data set class to create the internal indices for the model.  We then fit the model with the training set, and score with the test set.  

In [135]:
# import lightfm library and auc_score function
from lightfm import LightFM # model
from lightfm.evaluation import auc_score

C:\Users\pseudo_bounce\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [136]:
#instantiate light fm model with 20 components (same as als model)
modelfm = LightFM(
    no_components=20,
    learning_rate=0.05,
    loss='warp',
    random_state=2019)

In [140]:
#import lightfm dataset class
from lightfm.data import Dataset

In [142]:
#fit dataset class with user/items while supplying no content features
dataset = Dataset()
dataset.fit(
    users, 
    songs,
    item_features=None, 
    user_features=None)

In [146]:
#fit the model on the training data
modelfm.fit(
    train,
    item_features=None,
    user_features=None, sample_weight=None,
    epochs=5, num_threads=4, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [247]:
#score the model on the test data
score = auc_score( 
        modelfm, test, 
        item_features=None, 
        user_features=None, 
        num_threads=4).mean()

In [251]:
print(f'Lightfm auc score: {score}')

Lightfm auc score: 0.9372848272323608


We can see that the standard lightfm model out-performs the ALS model by about 2% and the popularity model by 6%.  At this point we could try and improve the results by including user/item metadata to create a hybrid model, and grid-searching parameters to find the optimal values.  The hybrid model is beyond the scope of this work, but the project could be used as a starting point for further developement.